In [ ]:
import ezomero
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

conn = ezomero.connect(
    os.getenv("OMERO_USERNAME"), 
    os.getenv("OMERO_PASSWORD"), 
    os.getenv("OMERO_GROUP"),
    host=os.getenv("OMERO_HOST"),
    port=int(os.getenv("OMERO_PORT")),
    secure=os.getenv("OMERO_SECURE") == "True"
)


In [4]:
ezomero.post_map_annotation(conn, img, "Image uploaded via BioIO and ezomero")

TypeError: post_map_annotation() missing 2 required positional arguments: 'kv_dict' and 'ns'

In [ ]:
ns = 'omero/collection/'

d = {'name': 'My Collection',
     'type': 'collection'
}

annotation_id = ezomero.post_map_annotation(conn, "Image", 35394, d, ns)

10898

In [11]:
map_ann = ezomero.get_map_annotation_ids(conn, object_type="Image", object_id=35394)
map_ann

[10898]

In [14]:
#map to another image
label_img_id = 35395
label_image = conn.getObject("Image", label_img_id)

annotation = conn.getObject("Annotation", map_ann[0])  
  
# Link the annotation to the image  
label_image.linkAnnotation(annotation)

<MapAnnotationWrapper id=10898>